<a href="https://colab.research.google.com/github/mrhamedani/LLM-Agents/blob/main/12_LangSmith_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tracing and Evaluating LLMs with LangSmith.Medical (Agent with LangSmith)

Models: OpenAI

Colab Environment: CPU-High RAM.

Keys:
* Tracing LLMs
* LangSmith
* Agents.




In this notebook, we will explore how to trace the different calls that occur in a LangChain Agent using LangSmith. We will use a familiar agent, employed in the LangChain section of the course, where a RAG system with medical information was created.

So, not only will we observe the traces of the agent, but we will also examine the traces of the retriever. Additionally, we'll inspect the query sent to the vectorial database and the returned results.
__________

In [1]:
!pip install -q langchain==0.3.0
!pip install -q langchain-community==0.3.0
!pip install -q langchainhub==0.1.21
!pip install -q datasets==3.0.0
!pip install -q chromadb==0.5.5
!pip install -q llama-cpp-python

In [ ]:
!wget -c https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf -O mistral.gguf

In [ ]:
import pandas as pd
from datasets import Dataset

data_dict = {
    "Question": [
        "What are the symptoms of diabetes?",
        "How to treat high blood pressure?",
        "Is headache a symptom of COVID-19?",
        "What causes anemia?",
        "What is the treatment for asthma?",
        "What are the side effects of chemotherapy?",
        "How is a heart attack diagnosed?",
        "What are common allergy symptoms?",
        "What causes chronic fatigue?",
        "How is depression treated?",
        "What is the function of the liver?",
        "What are symptoms of kidney failure?",
        "How can I prevent osteoporosis?",
        "What is a migraine and how is it treated?",
        "What are symptoms of a urinary tract infection (UTI)?",
        "What are the stages of cancer?",
        "Can high cholesterol be reversed?",
        "What are the signs of a stroke?",
        "What are the early symptoms of HIV?",
        "How is pregnancy confirmed medically?",
        "What causes acne in teenagers?",
        "What are the risks of obesity?",
        "What is the flu vaccine and who should get it?",
        "How is arthritis treated?",
        "What is the role of insulin in the body?",
        "What are symptoms of a thyroid disorder?",
        "How does smoking affect the lungs?",
        "What is eczema and how can it be managed?",
        "Can anxiety cause physical symptoms?",
        "How is ADHD diagnosed in children?"
    ],
    "Answer": [
        "Common symptoms of diabetes include increased thirst, frequent urination, and fatigue.",
        "Treatment includes lifestyle changes and medications such as ACE inhibitors and beta blockers.",
        "Yes, headache can be a symptom of COVID-19, especially with fever and fatigue.",
        "Anemia can be caused by iron deficiency, chronic disease, or blood loss.",
        "Asthma is treated with bronchodilators, corticosteroids, and avoiding triggers.",
        "Side effects of chemotherapy include nausea, fatigue, hair loss, and infection risk.",
        "A heart attack is diagnosed through ECG, blood tests, and sometimes imaging.",
        "Common allergy symptoms include sneezing, itchy eyes, runny nose, and skin rashes.",
        "Chronic fatigue can result from viral infections, stress, or autoimmune disorders.",
        "Depression is treated through therapy, medications, and lifestyle changes.",
        "The liver detoxifies blood, produces bile, and stores energy in the form of glycogen.",
        "Symptoms of kidney failure include swelling, fatigue, confusion, and reduced urine output.",
        "Prevention includes calcium and vitamin D intake, weight-bearing exercise, and no smoking.",
        "Migraines are intense headaches often treated with rest, medications, and avoiding triggers.",
        "Symptoms include burning urination, frequent urge to urinate, and lower abdominal pain.",
        "Cancer is staged from 0 to 4 based on tumor size, lymph node involvement, and metastasis.",
        "Yes, with a healthy diet, exercise, and sometimes medication, cholesterol can improve.",
        "Stroke signs include sudden weakness, confusion, slurred speech, and loss of balance.",
        "Early HIV symptoms may include fever, sore throat, rash, and swollen glands.",
        "Pregnancy is confirmed with blood/urine tests detecting hCG and ultrasound imaging.",
        "Teen acne is mainly caused by hormonal changes, excess oil, and bacteria.",
        "Obesity increases the risk of diabetes, heart disease, stroke, and certain cancers.",
        "The flu vaccine protects against influenza viruses and is recommended annually.",
        "Arthritis treatment involves anti-inflammatory drugs, physical therapy, and joint care.",
        "Insulin helps cells absorb glucose from blood for energy or storage.",
        "Symptoms include weight change, fatigue, hair thinning, and mood swings.",
        "Smoking damages lung tissue, reduces oxygen intake, and raises cancer risk.",
        "Eczema causes dry, itchy skin and is managed with moisturizers and steroids.",
        "Yes, anxiety can cause symptoms like chest pain, nausea, and dizziness.",
        "ADHD is diagnosed with behavior assessments, questionnaires, and clinical interviews."
    ]
}


df = pd.DataFrame(data_dict)

df.head()


In [ ]:
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

df_loader = DataFrameLoader(df, page_content_column="Answer")
docs = df_loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1250, chunk_overlap=100)
texts = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(texts, embedding=embeddings)

from langchain.chains import RetrievalQA
retriever = db.as_retriever()

from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path="./mistral.gguf",
    temperature=0.7,
    max_tokens=512,
    top_p=0.95,
    n_ctx=2048,
    verbose=True,
)

qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")


In [11]:
# LangSmith
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "mistral"


In [13]:
!pip install -q duckduckgo-search

from duckduckgo_search import DDGS

def duckduckgo_search(query: str) -> str:
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=3))
        if results:
            return "\n".join([r["body"] for r in results])
        return "No result found."

# تست
duckduckgo_search("What is hypertension?")



'Hypertension is a blood pressure reading of 130/80 mm Hg or higher. It can cause serious health problems and often has no symptoms. Learn about the causes, risk factors, diagnosis and treatment of hypertension.\nHypertension is when the force of blood pushing against your artery walls is too high. It can cause serious complications like heart attack and stroke. Learn how to check, prevent and manage high blood pressure with lifestyle changes and medications.\nHigh blood pressure, or hypertension, is when the force of blood pushing against the walls of your blood vessels is too high. It can lead to serious health problems such as heart attack and stroke. Learn about the blood pressure categories, how to measure and control your blood pressure, and the consequences of high blood pressure.'

In [14]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
You are an intelligent medical assistant designed to help users answer medical questions.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}
""")


In [ ]:
from langchain.agents import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import hub

# رپ‌کردن ابزار Medical KB
def medical_qa_tool(input_text):
    return f"[Medical KB]: {qa.run(input_text)}"

# رپ‌کردن ابزار جست‌وجوی وب
def duckduckgo_tool(input_text):
    results = duckduckgo_search(input_text)
    return f"[DuckDuckGo]: {results}"

tools = [
    Tool(
        name="Medical KB",
        func=medical_qa_tool,
        description="useful for answering medical questions by querying the medical knowledge base. Input should be a search query string.."
    ),
    Tool(
    name="DuckDuckGo Search",
    func=duckduckgo_tool,
    description="Use this tool to search the web anonymously without API keys."
)

]

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=3,
    return_messages=True
)


from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools= tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    max_iterations=15,
    handle_parsing_errors=True
)


In [17]:
# تست ایجنت با دو ابزار مشخص‌شده
response1 = agent_executor.invoke({
    "input": "What are the symptoms of anemia, and are there any recent updates in its treatment?"
})


# چاپ خروجی‌ها
print(response1)




> Entering new AgentExecutor chain...


> Entering new None chain...


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'